<a href="https://colab.research.google.com/github/Adhira-Deogade/INFO-7374---Assignment-3/blob/master/Testing_GE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.13.1


In [3]:
from google.colab import files
up = files.upload()

In [0]:
data = pd.read_csv("justcsv.csv")

In [0]:
data.drop(columns="Unnamed: 0", inplace=True)

In [6]:

# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 1319
Test size: 330


In [0]:
train_posts = data.text[:train_size]
train_tags = data.sentiment[:train_size]

test_posts = data.text[train_size:]
test_tags = data.sentiment[train_size:]

In [0]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)

In [9]:
data.head()

,text,sentiment
0,good day ladi gentlemen welcom tesla inc q fin...,neutral
1,sure think model littl bit slower sure,negative
2,well need bring shanghai factori onlin think b...,negative
3,elig sound like go reduc china come year reall...,neutral
4,demand demand model insan high inhibitor affor...,positive


In [12]:
data.dtypes

text         object
sentiment    object
dtype: object

In [14]:

# tokenize.fit_on_texts(train_posts) # only fit on train
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features = 2000, min_df = 3, max_df = 0.6)
# X = matrix.fit_transform(data).toarray()
# x_train = tokenize.texts_to_matrix(train_posts).astype(np.float32) # model expects float32
# x_test = tokenize.texts_to_matrix(test_posts).astype(np.float32)
x_train = matrix.fit_transform(train_posts).toarray()
x_test = matrix.fit_transform(test_posts).toarray()

ValueError: ignored

In [0]:

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [0]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes).astype(np.float32)
y_test = keras.utils.to_categorical(y_test, num_classes).astype(np.float32)

In [0]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [0]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 2

In [0]:

# Build the model
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, input_shape=(max_words,), name="posts"))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:

# Convert Keras model to estimator
estimator_model = keras.estimator.model_to_estimator(keras_model=model)

In [0]:
# Define input fn
def input_function(features, labels=None, shuffle=False):
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"posts_input": features}, # See the accompanying blog post for explanation on naming
        y=labels,
        shuffle=shuffle
    )
    return input_fn

In [0]:
estimator_model.train(input_fn=input_function(x_train, y_train, shuffle=True))

In [0]:
# Evaluate the accuracy of our trained model
score = estimator_model.evaluate(input_function(x_test, labels=y_test))
print(score)

In [0]:
y_softmax = model.predict(x_test)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [0]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [0]:
text_labels = encoder.classes_ 

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(24,20))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix")
plt.show()

In [0]:
from sklearn.metrics import f1_score

In [0]:
f1_score(y_test_1d, y_pred_1d, average='macro')